In [24]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [25]:
df= pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [27]:
#preprocess data
x= df.drop(columns=['Outcome'])
y= df['Outcome']
x_train, x_test, y_train,y_test= train_test_split(x,y,test_size=0.2, random_state=42)
scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

In [28]:
#TensorFlow/Keras Model
model= Sequential()
model.add(Input(shape=(x_train.shape[1],)))
model.add(Dense(12,activation='relu'))
model.add(Dense(8,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [29]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=10,batch_size=10,verbose=1)

Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.5066 - loss: 0.6999
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6742 - loss: 0.6526
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7187 - loss: 0.5955
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7383 - loss: 0.5286
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7828 - loss: 0.4761
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7556 - loss: 0.4965
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7473 - loss: 0.4799
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7858 - loss: 0.4579
Epoch 9/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7533 - loss: 0.4846
Epoch 10/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8026 - loss: 0.4407


In [30]:
y_pred= model.predict(x_test)
y_pred=(y_pred>0.5)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


In [31]:
accuracy= accuracy_score(y_test,y_pred)
print(accuracy)     

0.7337662337662337


In [33]:
###Pytorch

In [34]:
#convert data to pytorch tensors
x_train_tensor= torch.tensor(x_train,dtype=torch.float32)
x_test_tensor= torch.tensor(x_test,dtype=torch.float32)
y_train_tensor= torch.tensor(y_train.values,dtype=torch.float32).view(-1,1)
y_test_tensor= torch.tensor(y_test.values, dtype=torch.float32).view(-1,1)

In [35]:
#define model
model= nn.Sequential(
    nn.Linear(x_train.shape[1],12),nn.ReLU(),
    nn.Linear(12,8),nn.ReLU(),
    nn.Linear(8,1),nn.Sigmoid()
)

In [36]:
#train the model
optimizer=optim.Adam(model.parameters(),lr=0.001)
critreion= nn.BCELoss()

for _ in range(10):
    optimizer.zero_grad()
    loss= critreion(model(x_train_tensor),y_train_tensor)
    loss.backward()
    optimizer.step()

In [37]:
# Evaluate the model
with torch.no_grad():
    torch_y_pred = (model(x_test_tensor) > 0.5).float()  # Make sure to cast the output to float
    accuracy = (torch_y_pred.eq(y_test_tensor).sum().item()) / y_test_tensor.size(0)  # Calculate accuracy

print(f'Accuracy: {accuracy}')


Accuracy: 0.38961038961038963
